In [5]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import time
import re

# 設定
BASE_URL = 'https://github.com/orgs/google/repositories?type=all'
DB_NAME = 'github_repos.db'
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# --- ヘルパー関数: スター数を数値に変換 ---
def parse_stars_to_int(star_text):
    if not star_text: return 0
    try:
        if 'k' in star_text:
            return int(float(star_text.replace('k', '')) * 1000)
        return int(float(star_text.replace(',', '')))
    except ValueError:
        return 0

# --- メイン処理 ---
def scrape_and_insert_directly():
    # 1. データベース接続とテーブル作成
    conn = sqlite3.connect(DB_NAME)
    cur = conn.cursor()
    
    # 既にテーブルがある場合は削除して作り直す（リセット用）
    cur.execute('DROP TABLE IF EXISTS repositories')
    
    # テーブル作成 (SQLの定義通り)
    # 順番: id, name, primary_language, stars
    cur.execute('CREATE TABLE repositories (id INT, name TEXT, primary_language TEXT, stars INT);')
    conn.commit()
    print(f"🗄️ データベース '{DB_NAME}' を作成し、接続しました。")

    page_num = 1
    total_repos = 0
    previous_page_first_repo = None
    
    try:
        while True:
            current_url = f"{BASE_URL}&page={page_num}"
            print(f"\n📄 {page_num}ページ目を処理中... ({current_url})")
            
            response = requests.get(current_url, headers=HEADERS)
            if response.status_code != 200:
                print(f"❌ エラー: ステータスコード {response.status_code}")
                break
            
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # --- 要素の特定ロジック ---
            repo_items = []
            repo_list_container = soup.find('div', class_=lambda c: c and 'ListView_module__list-view-container' in c)
            if repo_list_container:
                repo_items = repo_list_container.find_all('li')
            
            if not repo_items:
                # フォールバック
                all_li_tags = soup.find_all('li')
                for li in all_li_tags:
                    if li.find('a', class_='Title-module__anchor--GmXUE Title-module__inline--oM0P7'):
                        repo_items.append(li)

            if not repo_items:
                print("🏁 アイテムが見つかりません。終了します。")
                break
            
            # 無限ループ防止
            check_name = 'N/A'
            check_tag = repo_items[0].find('a', class_='Title-module__anchor--GmXUE Title-module__inline--oM0P7')
            if check_tag and check_tag.find('span'): check_name = check_tag.find('span').text.strip()
            
            if check_name == previous_page_first_repo:
                print("🏁 前ページと同じ内容のため終了します。")
                break
            previous_page_first_repo = check_name
            
            # --- データ抽出と整形 ---
            page_data_for_db = []
            
            for i, item in enumerate(repo_items):
                repo_id = total_repos + i + 1
                
                # 名前
                repo_name = 'N/A'
                name_tag = item.find('a', class_='Title-module__anchor--GmXUE Title-module__inline--oM0P7')
                if name_tag and name_tag.find('span'):
                    repo_name = name_tag.find('span').text.strip()
                
                # 言語
                language = 'N/A'
                lang_circle = item.find('div', class_=lambda c: c and 'LanguageCircle-module__LanguageCircleIndicator' in c)
                if lang_circle:
                    lang_span = lang_circle.find_next_sibling('span')
                    if lang_span: language = lang_span.text.strip()
                
                # スター数 (ここで数値化！)
                stars_int = 0
                star_tag = item.find('a', href=lambda h: h and '/stargazers' in h)
                if star_tag:
                    raw_text = star_tag.get_text(strip=True)
                    stars_int = parse_stars_to_int(raw_text) # ここで変換関数を使う
                
                if repo_name != 'N/A':
                    # データベースの列の順番に合わせる！
                    # (id, name, primary_language, stars)
                    page_data_for_db.append((repo_id, repo_name, language, stars_int))
            
            # --- データベースへ一括挿入 ---
            if page_data_for_db:
                sql = "INSERT INTO repositories (id, name, primary_language, stars) VALUES (?, ?, ?, ?);"
                cur.executemany(sql, page_data_for_db)
                conn.commit() # この時点で保存確定
                
                count = len(page_data_for_db)
                total_repos += count
                print(f"✅ DBに {count} 件追加 (合計: {total_repos}件)")
                
                page_num += 1
                print("💤 1秒待機...")
                time.sleep(1)
            else:
                print("⚠️ データが空でした。")
                break
                
    except Exception as e:
        print(f"❌ 予期せぬエラー: {e}")
    finally:
        conn.close()
        print("🔒 データベース接続を閉じました。処理完了！")

if __name__ == '__main__':
    scrape_and_insert_directly()

🗄️ データベース 'github_repos.db' を作成し、接続しました。

📄 1ページ目を処理中... (https://github.com/orgs/google/repositories?type=all&page=1)
✅ DBに 30 件追加 (合計: 30件)
💤 1秒待機...

📄 2ページ目を処理中... (https://github.com/orgs/google/repositories?type=all&page=2)
✅ DBに 30 件追加 (合計: 60件)
💤 1秒待機...

📄 3ページ目を処理中... (https://github.com/orgs/google/repositories?type=all&page=3)
✅ DBに 30 件追加 (合計: 90件)
💤 1秒待機...

📄 4ページ目を処理中... (https://github.com/orgs/google/repositories?type=all&page=4)
✅ DBに 30 件追加 (合計: 120件)
💤 1秒待機...

📄 5ページ目を処理中... (https://github.com/orgs/google/repositories?type=all&page=5)
✅ DBに 30 件追加 (合計: 150件)
💤 1秒待機...

📄 6ページ目を処理中... (https://github.com/orgs/google/repositories?type=all&page=6)
✅ DBに 30 件追加 (合計: 180件)
💤 1秒待機...

📄 7ページ目を処理中... (https://github.com/orgs/google/repositories?type=all&page=7)
✅ DBに 30 件追加 (合計: 210件)
💤 1秒待機...

📄 8ページ目を処理中... (https://github.com/orgs/google/repositories?type=all&page=8)
✅ DBに 30 件追加 (合計: 240件)
💤 1秒待機...

📄 9ページ目を処理中... (https://github.com/orgs/google/repositories?type=

In [8]:
# データ確認
path = ''
db_name = 'github_repos.db'

try:
    conn = sqlite3.connect(path + db_name)
    cur = conn.cursor()

    #SQL文の作成 
    sql = "SELECT * FROM  repositories; "

    #SQl文の実行
    cur.execute(sql)

except sqlite3.Error as e:
    print('エラーが発生しました',e)
    
else:
    for row in cur:
        id, name, primary_language, stars = row
        print(name, primary_language, stars)    
    
finally:
    conn.close()


crubit C++ 924
koladata C++ 27
perfetto C++ 5000
heir C++ 610
wasefire Rust 129
dwh-migration-tools Java 54
osv-scalibr Go 538
zerocopy Rust 2100
auto Java 11000
yggdrasil-decision-forests C++ 622
ink C++ 49
oss-fuzz-vulns Python 166
merchant-api-samples Java 10
secure-aggregation Rust 5
adk-java Java 973
ksp Kotlin 3300
XNNPACK C 2200
skia-buildbot Go 158
xls C++ 1400
clusterfuzz Python 5500
skia C++ 10000
garf Python 11
or-tools C++ 13000
go-github Go 11000
conscrypt Java 1400
adk-go Go 4900
site-kit-wp JavaScript 1300
orbax Python 455
adk-python Python 16000
angle C++ 3800
nomulus Java 1800
nearby C++ 888
device-infra Java 58
selinux-policy-languages N/A 14
aarch64-esr-decoder Rust 99
kotlin-fhirpath Kotlin 7
dawn C++ 781
tunix Python 1900
desugar_jdk_libs Java 389
tcmalloc C++ 5000
meridian Python 1200
filonov Python 11
osv-scanner Go 8100
boringssl C++ 2100
brax Jupyter Notebook 2900
cel-java Java 227
ground-android Kotlin 272
go-attestation Go 405
openhtf Python 614
qwix Python 6